In [ ]:
# Importamos las librerías que necesitamos

# Librerías de extracción de datos
# -----------------------------------------------------------------------

# Importaciones:
# Beautifulsoup
from bs4 import BeautifulSoup

# Requests
import requests

import pandas as pd
import numpy as np

from time import sleep
import random
import datetime
from tqdm import tqdm
import re
import random as rand

# Importar librerías para automatización de navegadores web con Selenium
# -----------------------------------------------------------------------
from selenium import web  # Selenium es una herramienta para automatizar la interacción con navegadores web.
from web_manager.chrome import ChromeManager  # ChromeManager gestiona la instalación del controlador de Chrome.
from selenium.web.common.keys import Keys  # Keys es útil para simular eventos de teclado en Selenium.
from selenium.web.support.ui import Select  # Select se utiliza para interactuar con elementos <select> en páginas web.
from selenium.web.support.ui import WebWait
from selenium.web.support import expected_conditions as EC
from selenium.common.exceptions import NoSuchElementException # Excepciones comunes de selenium que nos podemos encontrar 

In [ ]:
def obtener_tabla(texto_tabla, encabezados, localidad):

    lista_tabla = texto_tabla.split("\n")
    lugar_de_division = lista_tabla.index('Max Avg Min') - 1
    tabla_final = []
    for i in range(1, lugar_de_division+1):
        tabla_final.append(lista_tabla[i::lugar_de_division])

    df_ini = pd.DataFrame(tabla_final)
    df_final = pd.DataFrame()
    for j in range(df_ini.shape[1]):
        df_inter = df_ini[j].str.split(" ", expand=True)
        df_final = pd.concat([df_final, df_inter], axis=1)

    df_final.columns = df_final.iloc[0]

    mes = df_final.iloc[0,0]
    nuevas_columnas = [encabezados[0]]

    for titulo in encabezados[1:-1]:
        for i in range(1,4):
            patron_regex = r"\(.*?\)" # buscamos cualquier texto que esté entre paréntesis, incluyendo los paréntesis.
            nuevas_columnas.append(re.sub(patron_regex, " ", titulo) + df_final.columns[i])

    nuevas_columnas.append("Precipitaciones")

    df_final.columns = nuevas_columnas

    df_final.drop(index=0, inplace=True)
    df_final.reset_index(drop=True, inplace=True)
    df_final.insert(column="Mes", loc=0, value=mes)
    df_final.insert(column="Localidad", loc=0, value=localidad)
    return df_final

df = pd.read_csv("datos/coordenadas_municipios.csv")

municipios = df["Municipio"].unique()
municipios_aleatorios = rand.choices(municipios, c=5)


